In [1]:
# 忽略各模块的警告信息
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 加载所需使用的模块
from datetime import datetime

from vnpy.trader.datafeed import get_datafeed
from vnpy.trader.database import get_database, DB_TZ
from vnpy.trader.constant import Interval
from vnpy.trader.object import BarData, HistoryRequest
from vnpy.trader.utility import extract_vt_symbol
from vnpy.trader.setting import SETTINGS
from akshare import stock_feature as sf

# 配置数据服务
SETTINGS["datafeed.name"] = "akshare"            # 可以根据自己的需求选择数据服务：rqdata/xt/wind等
SETTINGS["datafeed.username"] = "license"       # RQData的用户名统一为“license”这个字符串
SETTINGS["datafeed.password"] = "123456"        # 这里需要替换为你购买或者申请试用的RQData数据license

# 配置数据库
SETTINGS["database.name"] = "taos"              # 可以根据自己的需求选择数据库，这里使用的是TDengine
SETTINGS["database.database"] = "vnpy"
SETTINGS["database.host"] = "127.0.0.1"
SETTINGS["database.port"] = 6030
SETTINGS["database.user"] = "root"
SETTINGS["database.password"] = "taosdata"

# 创建对象实例
datafeed = get_datafeed()

# database = get_database()

# 要下载数据的起止时间
start = datetime(2025, 5, 30, tzinfo=DB_TZ)
end = datetime(2025, 7, 4, tzinfo=DB_TZ)

datafeed: <vnpy_akshare.akshre_feed.AKShareDataFeed object at 0x103cdfcd0>


In [3]:
# 要下载数据的合约代码
vt_symbols = [
    "000001.SZSE",

    "IF2501.CFFEX",

    # "IH2501.CFFEX",

    # "IC2501.CFFEX",

    # "IM2501.CFFEX",
]

In [4]:
# 遍历列表执行下载
for vt_symbol in vt_symbols:
    # 拆分合约代码和交易所
    symbol, exchange = extract_vt_symbol(vt_symbol)

    # 创建历史数据请求对象
    req: HistoryRequest = HistoryRequest(
        symbol=symbol,
        exchange=exchange,
        start=start,
        end=end,
        interval=Interval.HOUR
    )

    # 从数据服务下载数据
    bars: list[BarData] = datafeed.query_bar_history(req)

    if bars:
        print(bars)
        # database.save_bar_data(bars)
        print(f"下载数据成功：{vt_symbol}，总数据量：{len(bars)}")
    else:
        print(f"下载数据失败：{vt_symbol}")

req: HistoryRequest(symbol='000001', exchange=<Exchange.SZSE: 'SZSE'>, start=datetime.datetime(2025, 5, 30, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Shanghai')), end=datetime.datetime(2025, 7, 4, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Shanghai')), interval=<Interval.HOUR: '1h'>), df:                      时间     open    close     high      low   涨跌幅    涨跌额  \
0   2025-05-30 10:30:00  1748.73  1747.48  1754.98  1744.98  0.00   0.00   
1   2025-05-30 11:30:00  1747.48  1757.48  1758.73  1747.48  0.57  10.00   
2   2025-05-30 14:00:00  1757.48  1758.73  1759.98  1754.98  0.07   1.25   
3   2025-05-30 15:00:00  1758.73  1759.98  1762.48  1757.48  0.07   1.25   
4   2025-06-03 10:30:00  1757.48  1787.48  1788.73  1756.23  1.56  27.50   
..                  ...      ...      ...      ...      ...   ...    ...   
91  2025-07-02 15:00:00  1906.50  1900.25  1907.75  1900.25 -0.33  -6.25   
92  2025-07-03 10:30:00  1901.50  1900.25  1912.75  1895.25  0.00   0.00   
93  2025-07-03 11:30:00  1901